In [1]:
import math
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
from sificc_lib import utils, Event, Simulation, root_files, AI

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.float_format = '{:,.3f}'.format

%matplotlib inline

In [2]:
# finding the physical limit in accordance to the matching criteria
# used in my thesis work
simulation = Simulation(root_files.HPC_LLR)

c_compt=0 #counter for the ideal compton events
c_match=0 #counter for the matches

# iterate through the simulation events
for event in simulation.iterate_events():
    #check if the event is valid and ideal compton
    if event.is_distributed_clusters and event.is_ideal_compton:
        c_compt+=1
        l_e_clust = set()
        l_p_clust = set()
        
        # main e cluster is chosen by the matching cluster within its uncertainties
        # if nothing is matching, the closest cluster is chosed
        if event.e_clusters_count == 1:
            idx = event._arg_matching_cluster(event.real_e_position)
        else:
            idx = event._arg_closest_cluster(event.real_e_position)
        l_e_clust.add(idx)

        # main p cluster is chosen by the matching cluster within its uncertainties
        # if nothing is matching, the closest cluster is chosed
        if event.p_clusters_count == 1:
            idx = event._arg_matching_cluster(event.real_p_position)
        else:
            idx = event._arg_closest_cluster(event.real_p_position)
        l_p_clust.add(idx)
            
        # check all the interaction positions of e against all the clusters
        # and assign the matching clusters to e clusters
        for point in event.real_e_position_all.tolist():
            for i in range(event.clusters_count):
                if utils.is_point_inside_cluster(point, event.clusters_position[i], 
                                                 event.clusters_position_unc[i]):
                    l_e_clust.add(i)
                    
        # check all the interaction positions of p against all the clusters
        # and assign the matching clusters to p clusters if it's not already
        # assigned to e clusters
        for point in event.real_p_position_all.tolist()[1:]:
            for i in range(event.clusters_count):
                if utils.is_point_inside_cluster(point, event.clusters_position[i], 
                                                 event.clusters_position_unc[i]):
                    if i not in l_e_clust:
                        l_p_clust.add(i)
                    
        l_e_clust = list(l_e_clust)
        l_p_clust = list(l_p_clust)
        
        # the position is defined by the first interaction cluster
        e_pos = event.clusters_position[l_e_clust[0]]
        p_pos = event.clusters_position[l_p_clust[0]]
        
        # the energy is defined by the sum of all matching clusters
        e_enrg = event.clusters_energy[l_e_clust].sum()
        p_enrg = event.clusters_energy[l_p_clust].sum()
        
        # check if the recunstructed clusters qualify for a match according to the
        # criteria used in my thesis work
        if np.abs(event.real_e_position.x - e_pos.x) <= 2.6 \
                and np.abs(event.real_e_position.y - e_pos.y) <= 20 \
                and np.abs(event.real_e_position.z - e_pos.z) <= 2.6 \
                and np.abs(event.real_p_position.x - p_pos.x) <= 2.6 \
                and np.abs(event.real_p_position.y - p_pos.y) <= 20 \
                and np.abs(event.real_p_position.z - p_pos.z) <= 2.6 \
                and np.abs(event.real_e_energy - e_enrg) <= (event.real_e_energy * .12) \
                and np.abs(event.real_p_energy - p_enrg) <= (event.real_p_energy * .12):
            c_match +=1

processing root file: 100%|████████████████████████████████| 854873/854873 [17:24<00:00, 818.13it/s]


In [3]:
print('using my threshold')
print('total matches:', c_match)
print('total comptons:', c_compt)
print('physical limit: {:.2f}%'.format(c_match/c_compt*100))

using my threshold
total matches: 7029
total comptons: 97607
physical limit: 7.20%
